In [2]:
# load the libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Using the folllowing for references
# https://machinelearningmastery.com/quick-and-dirty-data-analysis-with-pandas/


In [4]:
output_width = 1000
#output_width = 80 #//*** Normal Output width
pd.set_option("display.width", output_width)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [5]:
# Load the data into a dataframe
housing_train = pd.read_csv("../../data/raw/dsc630_project/train.csv",sep=",")
housing_test = pd.read_csv("../../data/raw/dsc630_project/test.csv",sep=",")
housing_shared = pd.read_csv("../../data/processed/House_Prediction_upd.csv",sep=",")
# housing_data_0801 = pd.read_csv("../../data/processed/House_Prediction_08012021.csv",sep=",")

In [6]:
print("Training set dimension :", housing_train.shape)
print("Test set dimension :", housing_test.shape)

Training set dimension : (1460, 81)
Test set dimension : (1459, 80)


In [ ]:
# Viewing the top 5 and the bottom 5 rows in the dataframe
housing_train

In [ ]:
# Checking the info on the dataframe for the training set
# This shows the number of columns with the data types 
# float64(3), int64(35), object(43)
housing_train.info()

In [ ]:
# Creating a subset of the data with selected columns
housing_train_sub = housing_train[['Id','MSSubClass','LotArea','LotShape','LotConfig','Condition1','HouseStyle','YearBuilt','RoofMatl','MasVnrType','ExterCond','BsmtCond','BsmtFinSF1','BsmtUnfSF','HeatingQC','1stFlrSF','GrLivArea','FullBath','KitchenAbvGr','Functional','GarageType','GarageCars','GarageCond','OpenPorchSF','ScreenPorch','Fence','MoSold','SalePrice']]

housing_train_sub

In [ ]:
# store columns with specific data type
integer_columns = housing_train.select_dtypes(include=['int64']).columns
float_columns = housing_train.select_dtypes(include=['float64']).columns
object_columns = housing_train.select_dtypes(include=['object']).columns
  
# display columns
print('\nint64 columns:\n',integer_columns)
print('\nfloat64 columns:\n',float_columns)
print('\nobject columns:\n',object_columns)

In [ ]:
# Getting the integer columns into a separate dataframe

print("Number of integer columns: ",len(integer_columns))
df_int = housing_train[integer_columns]
# df_int.shape
# This displays a table of detailed distribution information for each of the 35 attributes(integer) in our dataframe
# We can start reviewing the statistics and collect interesting facts about the feature set
# df_int.describe()
df_int.info()

In [ ]:
# To find the correlation among
# the columns using pearson method
df_int.corr(method ='pearson')

In [ ]:
corr = df_int.corr(method ='pearson')
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(housing_train_sub, title="Pandas Profiling Report")
profile.to_file(output_file='Pandas_profile_Report.html')

In [ ]:
# Feature distributions
# Reviewing the distribution of some attributes
# We can also look at the distribution of each attribute by discretization the values into buckets and review the frequency in each bucket as histograms.
df_int[['LotArea','YearBuilt']].hist();

In [ ]:
df_int[['BsmtFinSF1','BsmtUnfSF']].hist();

In [ ]:
# Feature - Feature Relationships
# explore the relationships between the attributes
# distribution of the interactions of each pair of attributes
df1 = df_int[['BsmtFinSF1','BsmtUnfSF']]
from pandas.plotting import scatter_matrix
scatter_matrix(df1, alpha=0.2, figsize=(6, 6), diagonal='kde')


In [ ]:
for col in ['BsmtFinSF1','BsmtUnfSF']:
    df1[col] = df1[col].astype('category')

df1.info()

In [ ]:
df_int[['BsmtFinType1','OverallCond']].hist();

In [ ]:
# explore the relationships between the attributes
# distribution of the interactions of each pair of attributes
from pandas.plotting import scatter_matrix
scatter_matrix(df_int, alpha=0.2, figsize=(6, 6), diagonal='kde')


In [ ]:
## See the values of the object data types
housing_train['LotFrontage'].value_counts()

In [ ]:
# + housing_train["LowQualFinSF"])

assert not (
    housing_train["GrLivArea"] != (housing_train["1stFlrSF"] + housing_train["2ndFlrSF"]  + housing_train["LowQualFinSF"]) 
        ).any()

In [ ]:
housing_train[['LotFrontage']].hist();

In [ ]:
sns.histplot(housing_train.LotFrontage, kde=True)
plt.title("Distribution of the Lot Frontage length of the houses")
plt.show() 

In [ ]:
# Number of missing data per column
# List of columns with missing values

column_with_nan = housing_train.columns[housing_train.isnull().any()]
for column in column_with_nan:
    print(column, housing_train[column].isnull().sum())

In [ ]:
# Top 5 columns with largest number of missing data
housing_train.isnull().sum().nlargest(5)

In [ ]:
### Planning to drop the following columns
### One of the major reason for that is as more than 50% of the rows are Null for those columns

# PoolQC
# MiscFeature
# Alley
# Fence
# Below is the list of columns to be dropped
cols_to_drop=['PoolQC','MiscFeature','Alley','Fence']
# Making a copy of the training dataframe to be worked on
housing_train_drop = housing_train
print("Dimension before drop:", housing_train_drop.shape)
housing_train_drop = housing_train_drop.drop(cols_to_drop,axis=1)
print("Dimension after drop:", housing_train_drop.shape)
print("Dimension of original dataframe:", housing_train.shape)


In [ ]:
housing_train_drop.shape

In [ ]:
# Draw vertical boxplot with the LotFrontage values
sns.set_theme(style="whitegrid")
ax = sns.boxplot(y="LotFrontage", data=housing_train)

In [ ]:
housing_train.groupby(['LotFrontage'])['Id'].count()

In [ ]:
#### Fill up the missing values for the below with mean values
# LotFrontage
housing_train['LotFrontage'] = housing_train['LotFrontage'].fillna(df['LotFrontage'].mean())

In [ ]:
#### Fill up the missing values for the below with mode
#### These are categorical values so taking the mode is because I want to replace with the 
#### most number of occurance of that value
# BsmtExposure
housing_train['BsmtExposure'].value_counts()

In [ ]:
housing_train['BsmtExposure']=housing_train['BsmtExposure'].fillna(housing_train['BsmtExposure'].mode()[0])

In [ ]:
housing_train['BsmtExposure'].value_counts()

In [ ]:
housing_train['BsmtExposure'].isnull().sum()

In [ ]:
housing_train['LotArea'].value_counts(sort=True)

In [ ]:
# Draw vertical boxplot with the LotFrontage values
sns.set_theme(style="whitegrid")
ax = sns.boxplot(y="LotArea", data=housing_train)

In [ ]:
# SHowing the distribution of the variable Year Built
sns.histplot(housing_train.LotArea, kde=True)
plt.title("Distribution of the Lot Area of the houses")
plt.show() 

In [ ]:
# Scatter Plot showing # of Bedrooms and saleprice

plt.scatter(housing_train['BedroomAbvGr'],housing_train['SalePrice'])
plt.xlabel('BedroomAbvGr', color='red')
plt.ylabel('SalePrice', color='red')
plt.title('# of Bedrooms Vs Sale Price', color='red')
plt.show()

In [ ]:
# SHowing the distribution of the variable Year Built
sns.histplot(housing_train.BedroomAbvGr, kde=True)
plt.title("Distribution of the Bedroom counts of the houses")
plt.show() 

In [ ]:
# Scatter Plot showing lot Area and saleprice

plt.scatter(housing_train['LotArea'],housing_train['SalePrice'])
plt.xlabel('LotArea', color='red')
plt.ylabel('SalePrice', color='red')
plt.title('Lot Area Vs Sale Price', color='red')
plt.show()

In [ ]:
# Scatter Plot showing OverallCond: Rates the overall condition of the house and saleprice

plt.scatter(housing_train['OverallCond'],housing_train['SalePrice'])
plt.xlabel('OverallCond', color='red')
plt.ylabel('SalePrice', color='red')
plt.title('Rates the overall condition Vs Sale Price', color='red')
plt.show()

In [ ]:
# subset of the dataframe 

housing_train_saleyr = housing_train[['Neighborhood','YearBuilt','MoSold','YrSold','SalePrice']]
print("Dimension of the dataframe:",housing_train_saleyr.shape)



In [ ]:
housing_train[['Neighborhood','YearBuilt','MoSold','YrSold','SalePrice']].value_counts()

In [ ]:
housing_train_saleyr[(housing_train.YearBuilt == 2004)]


In [ ]:
# SHowing the distribution of the variable Year Built
sns.histplot(housing_train_saleyr.YearBuilt, kde=True)
plt.title("Distribution of the Year Built of the houses")
plt.show() 

In [ ]:
# Showing the distribution of the variable OverallCond
sns.histplot(housing_train.OverallCond, kde=True)
plt.title("Distribution of the overall condition of the houses")
plt.show() 

In [ ]:

assert ( housing_train['GrLivArea']
    != (housing_train['1stFlrSF'] + housing_train['2ndFlrSF'] + housing_train['LowQualFinSF'])
)

In [ ]:
# housing_train[['GrLivArea','1stFlrSF','2ndFlrSF','LowQualFinSF']]

housing_train['LowQualFinSF'].value_counts()

In [ ]:
# Final List of columns
col_list =['MSSubClass','MSZoning','LotArea','Street','LotShape','LandContour','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','OverallQual','OverallCond','YearBuilt','YearRemodAdd','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','MasVnrArea','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','HeatingQC','CentralAir','BedroomAbvGr','KitchenAbvGr','KitchenQual','TotRmsAbvGrd','Functional','Fireplaces','FireplaceQu','GarageType','GarageFinish','GarageCars','GarageArea','GarageCond','PavedDrive','PoolArea','PoolQC','MiscFeature','MoSold','YrSold','SaleCondition']
len(col_list)

In [ ]:
# housing_final = housing_train[col_list]
housing_final.shape

In [ ]:
# Derived variables
# TotalSF   [GrLivArea = 1stFlrSF,2ndFlrSF,LowQualFinSF] [TotalBsmtSF=BsmtFinSF1, BsmtFinSF2, BsmtUnfSF]
# TotalBath [BsmtFullBath, BsmtHalfBath, FullBath, HalfBath]
# TotalPorchSF [OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, WoodDeckSF]

In [ ]:
# Calculate a variable for the total living area Total SF as this is the number communicated most often in housing ads

housing_final["TotalSF"] = housing_train["GrLivArea"] + housing_train["TotalBsmtSF"]

# housing_train["GrLivArea"]
# housing_train.info()

In [ ]:
housing_final["TotalBath"] = (
    housing_train["FullBath"] + 0.5 * housing_train["HalfBath"] + housing_train["BsmtFullBath"] + 0.5 * housing_train["BsmtHalfBath"]
)

In [ ]:
housing_final["TotalPorchSF"] = (
    housing_train["OpenPorchSF"] + housing_train["EnclosedPorch"] + housing_train["3SsnPorch"] + housing_train["ScreenPorch"] + housing_train["WoodDeckSF"]
)

In [ ]:
# Target Variable
target_var = pd.DataFrame(housing_train['SalePrice'])
print("Shape of the traget variable:", target_var.shape)
# showing the first 5 rows
target_var.head()

In [ ]:
# Saving the feature set till now
housing_final.to_csv("housing_final_feat.csv")

In [ ]:
# Saving the target set till now
target_var.to_csv("housing_final_tgt.csv")

In [ ]:
# Filtering out some features based on lot area size

# housing_train.head()
# lotareagr50 = housing_train[housing_train['LotArea']> 50000]  
housing_train[housing_train['LotArea']== 115149][['Id','LotArea','GrLivArea','TotalBsmtSF','SalePrice']]

In [ ]:
# Verifying the same data from the final data set
housing_final[housing_final['LotArea']== 115149][['TotalSF','TotalBath','TotalPorchSF']]

In [ ]:
# Verifying the target variable in the target data set
target_var.loc[706]

In [ ]:
# Getting the columns from the final feature list
housing_final.info()

In [ ]:
# PoolQC and MiscFeature have lots of Null values so it is better to drop those cols from the final feature list
# housing_final['PoolQC'].value_counts()
# housing_final['PoolQC'].isnull().count()
# housing_final['PoolQC'].value_counts(sort=True)
housing_final['PoolQC'].fillna('NULL').value_counts()

In [ ]:
# housing_final['PoolQC'].value_counts()
# housing_final['PoolQC'].isnull().count()
# housing_final['PoolQC'].value_counts(sort=True)
housing_final['MiscFeature'].fillna('NULL').value_counts()

In [ ]:
housing_final = housing_final.drop(columns=['PoolQC','MiscFeature'])

In [ ]:
# Saving the new feature sets
housing_final.to_csv("housing_final_feat.csv")

### Preparing data for modeling

In [ ]:
# Checking to see if any of the columns has null values
# There are no columns with null values

housing_shared.isnull().sum()

In [4]:
# Encoding categorical labels
# dat2["Sex"] = dat2["Sex"].map({"M": 0, "F": 1})
# dat2['Sex'].value_counts()

housing_shared['MSZoning'].value_counts()

RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64

In [6]:
housing_shared['LotShape'].value_counts()

Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64

In [ ]:
housing_shared['ExterCond'].value_counts()

In [8]:
housing_shared.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 54 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   LotConfig      1460 non-null   object 
 8   LandSlope      1460 non-null   object 
 9   Neighborhood   1460 non-null   object 
 10  Condition1     1460 non-null   object 
 11  Condition2     1460 non-null   object 
 12  BldgType       1460 non-null   object 
 13  HouseStyle     1460 non-null   object 
 14  OverallQual    1460 non-null   int64  
 15  OverallCond    1460 non-null   int64  
 16  YearBuilt      1460 non-null   int64  
 17  YearRemodAdd   1460 non-null   int64  
 18  RoofStyl

In [23]:
# housing_shared_sub=housing_shared[['MSSubClass','MSZoning','LotShape','ExterCond']]
housing_shared_sub=housing_shared.loc[:, ('MSSubClass','MSZoning','LotShape','ExterCond')]

In [24]:
housing_shared_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   MSSubClass  1460 non-null   int64 
 1   MSZoning    1460 non-null   object
 2   LotShape    1460 non-null   object
 3   ExterCond   1460 non-null   object
dtypes: int64(1), object(3)
memory usage: 45.8+ KB


In [27]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
lotshape_encoded = le.fit_transform(housing_shared_sub['LotShape'])
ExterCond_encoded = le.fit_transform(housing_shared_sub['ExterCond'])
# type(lotshape_encoded)

housing_shared_sub['lotshape_encoded'] = lotshape_encoded
housing_shared_sub['ExterCond_encoded'] = ExterCond_encoded

In [1]:
housing_shared_sub.head()

NameError: name 'housing_shared_sub' is not defined

https://www.pluralsight.com/guides/preparing-data-modeling-scikit-learn

https://codefires.com/how-convert-categorical-data-numerical-data-python/